In [1]:
from utils.genral import get_precedients,get_precedients_triplets,get_precedients_anon,get_metadata
from utils.langchain import parser
import tqdm
import pickle

with open('small_precedents_ids.pkl', 'rb') as f:
    small_precedents_ids = pickle.load(f)
print("Number of small precedents: ", len(small_precedents_ids))

id2text = get_precedients()
good_response_triplet_folder = "dpo-triplets-annon/all"
bad_response_triplet_folder = "dpo-triplets/all"

Number of small precedents:  300


In [2]:
from transformers import (
    AutoTokenizer
)

base_model = "google/gemma-7b-it"

tokenizer = AutoTokenizer.from_pretrained(base_model)

/home/dhananjay/miniconda3/envs/gnn/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from utils.langchain import langchain_chat_template,parser
from utils.genral import clean_string
import json

prompts = []
chosens=[]
rejecteds=[]

for id in small_precedents_ids:
    

    chat_template = langchain_chat_template.format_messages(document=id2text[id])
    input_prompt = clean_string( chat_template[0].content + chat_template[1].content ) 
    chat=[
        {"role": "user", "content": input_prompt }
    ]
    prompt = tokenizer.apply_chat_template(chat, tokenize=False,add_generation_prompt=True)
    prompts.append(prompt)
    
    #GOOD REPSONE
    with open(f"{ good_response_triplet_folder }/{ id }.json", "r") as file:
        chosen = json.dumps( json.loads(file.read()) )
        chosens.append(chosen)

    #BAD REPSONE
    with open(f"{ bad_response_triplet_folder }/{ id }.json", "r") as file:
        rejected = json.dumps( json.loads(file.read()) )
        rejecteds.append(rejected)
    


In [4]:
from datasets import Dataset
dataset = Dataset.from_dict( {"prompt": prompts,"chosen": chosens, "rejected": rejecteds} )
dataset.save_to_disk("dataset/dpo")

Saving the dataset (1/1 shards): 100%|██████████| 300/300 [00:00<00:00, 27792.80 examples/s]


In [5]:
from datasets import Dataset
dataset = Dataset.load_from_disk("dataset/dpo")
print(dataset)
print(dataset[1]["chosen"])
print(dataset[1]["rejected"])

Dataset({
    features: ['prompt', 'chosen', 'rejected'],
    num_rows: 300
})
{"triplets": [{"head": {"value": "Respondents", "type": "Actor / Player"}, "relationship": "worked as", "tail": {"value": "Teachers in Department of Education", "type": "Service"}}, {"head": {"value": "Respondents", "type": "Actor / Player"}, "relationship": "availed", "tail": {"value": "Leave Travel Concession", "type": "Document / Artifact"}}, {"head": {"value": "Respondents", "type": "Actor / Player"}, "relationship": "drew amount instead of utilizing", "tail": {"value": "Leave Travel Concession", "type": "Document / Artifact"}}, {"head": {"value": "Recovery", "type": "Event"}, "relationship": "made in", "tail": {"value": "1984-86", "type": "Event"}}, {"head": {"value": "Respondents", "type": "Actor / Player"}, "relationship": "filed applications questioning", "tail": {"value": "Government's power of recovery", "type": "Event"}}, {"head": {"value": "Tribunal", "type": "Forums and Venues"}, "relationship":